In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier

In [3]:
accu_df = pd.read_csv("Datasets\synthetic_accural_data.csv")
redm_df = pd.read_csv("Datasets\synthetic_redeem_data.csv")

In [4]:
accu_df.head()

,transaction_id,customer_unique_id,transaction_date,transaction_type,points,partner_id,store_id,opening_balance,closing_balance,amount,is_anomaly,anomaly_types,country_code,activity_id,category_code,sku_category_code,item_code,quantity,created_at,updated_at,bill_date,business_date,rule_id,rule_type,payment_type,program_id,program_country,program_currency,base_currency,trans_currency,conversion_rate,amount_in_trans_currency,usd_conversion_rate,amount_in_usd,channel_code,point_expiry,funding_partner_id,point_type_id,calculated_amount
0,TXN00002862,0006417714fe4a78913920f39a03cdf7,2023-01-06 07:30:00.000000,earn,333.09,439,3896,0.00,333.09,2307.33,0,NaN,971,186,ELEC,SKU-A1,ITM6791,6,2023-01-05 07:30:00.000000,2023-01-08 07:30:00.000000,2023-01-04 07:30:00.000000,2023-01-04 07:30:00.000000,10,base,CASH,3,65,USD,USD,USD,0.6737,1554.45,0.0805,185.74,CPN,2023-06-05 07:30:00.000000,439,9,2217.93
1,TXN00002863,0006417714fe4a78913920f39a03cdf7,2023-01-08 06:19:00.000000,earn,314.54,439,3025,333.09,647.63,4863.34,0,NaN,91,232,ELEC,SKU-C3,ITM5485,7,2023-01-07 06:19:00.000000,2023-01-08 06:19:00.000000,2023-01-07 06:19:00.000000,2023-01-05 06:19:00.000000,43,base,CASH,18,91,INR,USD,INR,0.9181,4465.03,0.0969,471.26,CPN,2024-11-02 06:19:00.000000,439,13,4883.89
2,TXN00002864,0006417714fe4a78913920f39a03cdf7,2023-01-09 10:06:00.000000,earn,341.69,212,4950,647.63,989.32,644.58,0,NaN,65,429,HM45,SKU-B2,ITM5940,9,2023-01-06 10:06:00.000000,2023-01-10 10:06:00.000000,2023-01-07 10:06:00.000000,2023-01-06 10:06:00.000000,3,base,CARD,25,65,SGD,USD,SGD,0.6280,404.80,0.0687,44.28,POS,2023-12-29 10:06:00.000000,212,19,604.63
3,TXN00002866,0006417714fe4a78913920f39a03cdf7,2023-01-11 08:48:00.000000,earn,274.21,439,1144,660.26,934.47,3202.35,0,NaN,91,297,ELEC,SKU-B2,ITM5806,4,2023-01-08 08:48:00.000000,2023-01-13 08:48:00.000000,2023-01-08 08:48:00.000000,2023-01-08 08:48:00.000000,41,base,WALLET,13,971,AED,USD,AED,1.1400,3650.68,0.0767,245.62,POS,2024-10-30 08:48:00.000000,439,15,3627.60
4,TXN00002868,0006417714fe4a78913920f39a03cdf7,2023-01-13 08:15:00.000000,earn,350.79,439,1145,885.07,1235.86,4056.82,0,NaN,91,314,FASHX,SKU-A1,ITM2370,6,2023-01-11 08:15:00.000000,2023-01-13 08:15:00.000000,2023-01-10 08:15:00.000000,2023-01-10 08:15:00.000000,32,base,CARD,1,65,AED,USD,AED,1.2190,4945.26,0.0618,250.71,CPN,2023-12-27 08:15:00.000000,439,3,4557.73


In [5]:
redm_df.head()

,transaction_id,customer_unique_id,transaction_date,transaction_type,points,partner_id,store_id,opening_balance,closing_balance,amount,is_anomaly,anomaly_types,country_code,activity_id,category_code,sku_category_code,item_code,quantity,created_at,updated_at,bill_date,business_date,rule_id,rule_type,payment_type,program_id,program_country,program_currency,base_currency,trans_currency,conversion_rate,amount_in_trans_currency,usd_conversion_rate,amount_in_usd,channel_code,point_expiry,funding_partner_id,point_type_id,calculated_amount
0,TXN00002865,0006417714fe4a78913920f39a03cdf7,2023-01-10 06:39:00.000000,redeem,329.06,439,4201,989.32,660.26,2309.53,0,NaN,1,175,HM45,SKU-A1,ITM7103,8,2023-01-09 06:39:00.000000,2023-01-12 06:39:00.000000,2023-01-07 06:39:00.000000,2023-01-07 06:39:00.000000,36,base,CASH,27,65,INR,USD,INR,1.5147,3498.25,0.0457,105.55,CPN,2024-12-22 06:39:00.000000,439,18,2016.54
1,TXN00002867,0006417714fe4a78913920f39a03cdf7,2023-01-12 11:57:00.000000,redeem,49.40,439,3571,934.47,885.07,2484.20,0,NaN,1,356,HM45,SKU-B2,ITM7192,1,2023-01-10 11:57:00.000000,2023-01-13 11:57:00.000000,2023-01-10 11:57:00.000000,2023-01-11 11:57:00.000000,33,promo,UPI,23,971,INR,USD,INR,0.8328,2068.84,0.0509,126.45,CPN,2024-05-12 11:57:00.000000,439,7,2087.11
2,TXN00005307,002cc23bffa546f1b6135d696f7ee2f8,2023-01-09 19:05:00.000000,redeem,312.96,435,2112,459.79,146.83,1893.40,0,NaN,1,490,GROC123,SKU-C3,ITM7112,5,2023-01-07 19:05:00.000000,2023-01-09 19:05:00.000000,2023-01-06 19:05:00.000000,2023-01-08 19:05:00.000000,33,promo,WALLET,5,1,INR,USD,INR,1.1529,2182.90,0.0495,93.72,POS,2024-07-24 19:05:00.000000,435,7,1596.01
3,TXN00005310,002cc23bffa546f1b6135d696f7ee2f8,2023-01-13 21:39:00.000000,redeem,37.73,435,2473,740.86,703.13,4298.60,0,NaN,971,285,FASHX,SKU-A1,ITM3385,5,2023-01-12 21:39:00.000000,2023-01-13 21:39:00.000000,2023-01-12 21:39:00.000000,2023-01-10 21:39:00.000000,49,promo,CASH,24,1,SGD,USD,SGD,0.8989,3864.01,0.0794,341.31,POS,2023-10-18 21:39:00.000000,435,1,4166.84
4,TXN00005311,002cc23bffa546f1b6135d696f7ee2f8,2023-01-15 19:38:00.000000,redeem,87.01,435,1948,703.13,616.12,4442.77,0,NaN,65,105,GROC123,SKU-A1,ITM5996,6,2023-01-13 19:38:00.000000,2023-01-15 19:38:00.000000,2023-01-13 19:38:00.000000,2023-01-12 19:38:00.000000,39,base,CARD,19,1,USD,USD,USD,1.8908,8400.39,0.0416,184.82,CPN,2023-04-11 19:38:00.000000,435,14,3787.68


In [6]:
accu_df.dtypes

transaction_id                  str
customer_unique_id              str
transaction_date                str
transaction_type                str
points                      float64
partner_id                    int64
store_id                      int64
opening_balance             float64
closing_balance             float64
amount                      float64
is_anomaly                    int64
anomaly_types                   str
country_code                  int64
activity_id                   int64
category_code                   str
sku_category_code               str
item_code                       str
quantity                      int64
created_at                      str
updated_at                      str
bill_date                       str
business_date                   str
rule_id                       int64
rule_type                       str
payment_type                    str
program_id                    int64
program_country               int64
program_currency            

In [7]:
redm_df.dtypes

transaction_id                  str
customer_unique_id              str
transaction_date                str
transaction_type                str
points                      float64
partner_id                    int64
store_id                      int64
opening_balance             float64
closing_balance             float64
amount                      float64
is_anomaly                    int64
anomaly_types                   str
country_code                  int64
activity_id                   int64
category_code                   str
sku_category_code               str
item_code                       str
quantity                      int64
created_at                      str
updated_at                      str
bill_date                       str
business_date                   str
rule_id                       int64
rule_type                       str
payment_type                    str
program_id                    int64
program_country               int64
program_currency            

In [8]:
# Columns to drop
columns_to_drop = [
    'transaction_id',
    'customer_unique_id',
    'anomaly_types',
    'created_at',
    'updated_at',
    'bill_date',
    'business_date',
    'rule_id',
    'rule_type',
    'item_code',
    'sku_category_code',
    'category_code',
    'store_id',
    'activity_id',
    'funding_partner_id'
]

# Drop columns from accrual dataframe
print("Accrual DataFrame - Before drop:")
print(f"Shape: {accu_df.shape}")
print(f"Columns: {list(accu_df.columns)}")

accu_df = accu_df.drop(columns=columns_to_drop, errors='ignore')

print("\nAccrual DataFrame - After drop:")
print(f"Shape: {accu_df.shape}")
print(f"Columns: {list(accu_df.columns)}")

# Drop columns from redeem dataframe
print("\n" + "=" * 80)
print("Redeem DataFrame - Before drop:")
print(f"Shape: {redm_df.shape}")
print(f"Columns: {list(redm_df.columns)}")

redm_df = redm_df.drop(columns=columns_to_drop, errors='ignore')

print("\nRedeem DataFrame - After drop:")
print(f"Shape: {redm_df.shape}")
print(f"Columns: {list(redm_df.columns)}")

Accrual DataFrame - Before drop:
Shape: (8219, 39)
Columns: ['transaction_id', 'customer_unique_id', 'transaction_date', 'transaction_type', 'points', 'partner_id', 'store_id', 'opening_balance', 'closing_balance', 'amount', 'is_anomaly', 'anomaly_types', 'country_code', 'activity_id', 'category_code', 'sku_category_code', 'item_code', 'quantity', 'created_at', 'updated_at', 'bill_date', 'business_date', 'rule_id', 'rule_type', 'payment_type', 'program_id', 'program_country', 'program_currency', 'base_currency', 'trans_currency', 'conversion_rate', 'amount_in_trans_currency', 'usd_conversion_rate', 'amount_in_usd', 'channel_code', 'point_expiry', 'funding_partner_id', 'point_type_id', 'calculated_amount']

Accrual DataFrame - After drop:
Shape: (8219, 24)
Columns: ['transaction_date', 'transaction_type', 'points', 'partner_id', 'opening_balance', 'closing_balance', 'amount', 'is_anomaly', 'country_code', 'quantity', 'payment_type', 'program_id', 'program_country', 'program_currency', '

In [9]:
# Convert transaction_date to datetime and extract features
print("=" * 80)
print("ACCRUAL - Processing transaction_date")
print("=" * 80)
print(f"Before: {accu_df.shape}")
print(f"Columns with 'date' in name: {[col for col in accu_df.columns if 'date' in col.lower()]}")

# Convert to datetime
accu_df['transaction_date'] = pd.to_datetime(accu_df['transaction_date'])

# Extract features
accu_df['hour'] = accu_df['transaction_date'].dt.hour
accu_df['day_of_week'] = accu_df['transaction_date'].dt.dayofweek  # 0=Monday, 6=Sunday
accu_df['is_weekend'] = (accu_df['day_of_week'].isin([5, 6])).astype(int)

# Drop original column
accu_df = accu_df.drop(columns=['transaction_date'])

print(f"After: {accu_df.shape}")
print(f"New columns added: hour, day_of_week, is_weekend")
print(f"Columns: {list(accu_df.columns)}")

print("\n" + "=" * 80)
print("REDEEM - Processing transaction_date")
print("=" * 80)
print(f"Before: {redm_df.shape}")
print(f"Columns with 'date' in name: {[col for col in redm_df.columns if 'date' in col.lower()]}")

# Convert to datetime
redm_df['transaction_date'] = pd.to_datetime(redm_df['transaction_date'])

# Extract features
redm_df['hour'] = redm_df['transaction_date'].dt.hour
redm_df['day_of_week'] = redm_df['transaction_date'].dt.dayofweek  # 0=Monday, 6=Sunday
redm_df['is_weekend'] = (redm_df['day_of_week'].isin([5, 6])).astype(int)

# Drop original column
redm_df = redm_df.drop(columns=['transaction_date'])

print(f"After: {redm_df.shape}")
print(f"New columns added: hour, day_of_week, is_weekend")
print(f"Columns: {list(redm_df.columns)}")

print("\n" + "=" * 80)
print("Sample Data")
print("=" * 80)
print("Accrual:")
print(accu_df.head())
print("\nRedeem:")
print(redm_df.head())

ACCRUAL - Processing transaction_date
Before: (8219, 24)
Columns with 'date' in name: ['transaction_date']
After: (8219, 26)
New columns added: hour, day_of_week, is_weekend
Columns: ['transaction_type', 'points', 'partner_id', 'opening_balance', 'closing_balance', 'amount', 'is_anomaly', 'country_code', 'quantity', 'payment_type', 'program_id', 'program_country', 'program_currency', 'base_currency', 'trans_currency', 'conversion_rate', 'amount_in_trans_currency', 'usd_conversion_rate', 'amount_in_usd', 'channel_code', 'point_expiry', 'point_type_id', 'calculated_amount', 'hour', 'day_of_week', 'is_weekend']

REDEEM - Processing transaction_date
Before: (2364, 24)
Columns with 'date' in name: ['transaction_date']
After: (2364, 26)
New columns added: hour, day_of_week, is_weekend
Columns: ['transaction_type', 'points', 'partner_id', 'opening_balance', 'closing_balance', 'amount', 'is_anomaly', 'country_code', 'quantity', 'payment_type', 'program_id', 'program_country', 'program_currency

In [11]:
redm_df.head()

,transaction_type,points,partner_id,opening_balance,closing_balance,amount,is_anomaly,country_code,quantity,payment_type,program_id,program_country,program_currency,base_currency,trans_currency,conversion_rate,amount_in_trans_currency,usd_conversion_rate,amount_in_usd,channel_code,point_expiry,point_type_id,calculated_amount,hour,day_of_week,is_weekend
0,redeem,329.06,439,989.32,660.26,2309.53,0,1,8,CASH,27,65,INR,USD,INR,1.5147,3498.25,0.0457,105.55,CPN,2024-12-22 06:39:00.000000,18,2016.54,6,1,0
1,redeem,49.40,439,934.47,885.07,2484.20,0,1,1,UPI,23,971,INR,USD,INR,0.8328,2068.84,0.0509,126.45,CPN,2024-05-12 11:57:00.000000,7,2087.11,11,3,0
2,redeem,312.96,435,459.79,146.83,1893.40,0,1,5,WALLET,5,1,INR,USD,INR,1.1529,2182.90,0.0495,93.72,POS,2024-07-24 19:05:00.000000,7,1596.01,19,0,0
3,redeem,37.73,435,740.86,703.13,4298.60,0,971,5,CASH,24,1,SGD,USD,SGD,0.8989,3864.01,0.0794,341.31,POS,2023-10-18 21:39:00.000000,1,4166.84,21,4,0
4,redeem,87.01,435,703.13,616.12,4442.77,0,65,6,CARD,19,1,USD,USD,USD,1.8908,8400.39,0.0416,184.82,CPN,2023-04-11 19:38:00.000000,14,3787.68,19,6,1


In [12]:
accu_y = accu_df["is_anomaly"]
redm_y = redm_df["is_anomaly"]

In [14]:
accu_X = accu_df.drop(columns=["is_anomaly"])
redm_X = redm_df.drop(columns=["is_anomaly"])

In [16]:
redm_X.head()

,transaction_type,points,partner_id,opening_balance,closing_balance,amount,country_code,quantity,payment_type,program_id,program_country,program_currency,base_currency,trans_currency,conversion_rate,amount_in_trans_currency,usd_conversion_rate,amount_in_usd,channel_code,point_expiry,point_type_id,calculated_amount,hour,day_of_week,is_weekend
0,redeem,329.06,439,989.32,660.26,2309.53,1,8,CASH,27,65,INR,USD,INR,1.5147,3498.25,0.0457,105.55,CPN,2024-12-22 06:39:00.000000,18,2016.54,6,1,0
1,redeem,49.40,439,934.47,885.07,2484.20,1,1,UPI,23,971,INR,USD,INR,0.8328,2068.84,0.0509,126.45,CPN,2024-05-12 11:57:00.000000,7,2087.11,11,3,0
2,redeem,312.96,435,459.79,146.83,1893.40,1,5,WALLET,5,1,INR,USD,INR,1.1529,2182.90,0.0495,93.72,POS,2024-07-24 19:05:00.000000,7,1596.01,19,0,0
3,redeem,37.73,435,740.86,703.13,4298.60,971,5,CASH,24,1,SGD,USD,SGD,0.8989,3864.01,0.0794,341.31,POS,2023-10-18 21:39:00.000000,1,4166.84,21,4,0
4,redeem,87.01,435,703.13,616.12,4442.77,65,6,CARD,19,1,USD,USD,USD,1.8908,8400.39,0.0416,184.82,CPN,2023-04-11 19:38:00.000000,14,3787.68,19,6,1
